In [1]:
from fishsense_api_sdk.client import Client
from pathlib import Path
import io
import tarfile
import numpy as np
from fishsense_api_sdk.models.camera_intrinsics import CameraIntrinsics

In [2]:
INTRINSICS_PATH = Path("../intrinsics").absolute()

INTRINSICS_PATH.exists()

True

In [3]:
URL = "http://localhost:8000"

In [4]:
client = Client(URL)

In [5]:
cameras = await client.cameras.get()

len(cameras), cameras[0]

(11, Camera(id=1, serial_number='BJ6C69506', name='FSL-01'))

In [6]:
def _read_numpy_array(buffer: io.BufferedReader):
    with io.BytesIO(buffer.read()) as b:
        return np.load(b)


def read_numpy_array(file: tarfile.TarFile, name: str) -> np.ndarray:
    return _read_numpy_array(file.extractfile(name))


def read_camera_calibration(file_path: Path):
    with file_path.open("rb") as f:
        with tarfile.open(fileobj=f) as t:
            calibration_matrix = read_numpy_array(t, "_calibration_matrix.npy")
            distortion_coeffs = read_numpy_array(t, "_distortion_coefficients.npy")
    return calibration_matrix, distortion_coeffs

In [7]:
for camera in cameras:
    name = f"{camera.name}D"
    calibration_path = INTRINSICS_PATH / name / f"{name.lower()}-lens-raw.pkg"

    camera_matrix, distortion_coefficients = read_camera_calibration(calibration_path)

    camera_intrinsics = CameraIntrinsics(camera_matrix, distortion_coefficients, camera.id)

    await client.cameras.post_intrinsics(camera.id, camera_intrinsics)

In [8]:
camera_intrinsics._to_internal().model_dump()

{'id': None,
 'camera_matrix': [[2855.214458140686, 0.0, 1975.9404638306894],
  [0.0, 2881.198016892831, 1432.810609827023],
  [0.0, 0.0, 1.0]],
 'distortion_coefficients': [-0.10014835288043915,
  -0.11623296677283777,
  -0.0013661003169052597,
  0.0002913857827530041,
  0.10317095756502268],
 'camera_id': 11}